In [ ]:
### Build 2 linear layers for this task (train, testing, validate)
import torch.nn as nn
import torch.nn.functional as F

# Define the model
class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        # First layer: Transforms 10 input features to 6
        self.fc1 = nn.Linear(input_dim, 6)
        # Output layer: Transforms 6 features to 2 (binary classification)
        self.fc2 = nn.Linear(6, 2)
    
    def forward(self, x):
        # First layer with ReLU activation
        x = F.relu(self.fc1(x))
        # Output layer with softmax activation (for probabilities)
        x = F.softmax(self.fc2(x), dim=1)
        return x

# Define the input dimension based on your training data
input_dim = X_train.shape[-1]

# Instantiate the model
model = MyModel(input_dim)

# Check the model structure
print(model)

### Loss function and optimizor
import torch.optim as optim

# Define the loss function (categorical crossentropy in PyTorch)
criterion = nn.CrossEntropyLoss()

# Define the optimizer (you can use SGD, Adam, etc.)
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Learning rate = 0.01

### Convert data to PyTorch tensor
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.long)  # Long for class labels

X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.long)

### training loop
# Set the number of epochs and batch size
epochs = 50
batch_size = 5

# Prepare DataLoader (for batching)
train_data = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss and calculate accuracy
        running_loss += loss.item()

        # For accuracy calculation
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print epoch loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")